In [ ]:
import cv2
import torch
import numpy as np
import csv
import random
from os import listdir, makedirs
from os.path import isfile, join
import albumentations as A
import hickle
from IPython.display import clear_output
import time
from shutil import rmtree
from multiprocessing import Process

<h2>Global Variables</h2>

In [ ]:
src_dir = "/home/tyler/Documents/Data/PinData/videos/"
dest_dir = "/home/tyler/Documents/Data/PinData/PinVideosRotated/"
dest_dir_train = f"{dest_dir}train/"
dest_dir_val = f"{dest_dir}val/"

In [ ]:
IMAGE_WIDTH = 192
IMAGE_HEIGHT = 256

FRAME_CAP = 40
FPS = 100

In [ ]:
# rmtree(dest_dir, ignore_errors=True)

In [ ]:
makedirs(dest_dir, exist_ok=True)
makedirs(dest_dir_train, exist_ok=True)
makedirs(dest_dir_val, exist_ok=True)

In [ ]:
transform = A.Compose([
    A.LongestMaxSize(IMAGE_HEIGHT, always_apply=True),
    # A.GaussianBlur(),
    # A.ColorJitter(),
    # A.GaussNoise(),
    A.Normalize(always_apply=True),
    A.ToFloat(always_apply=True)
])

In [ ]:
transform_preview = A.Compose([
    A.LongestMaxSize(IMAGE_HEIGHT, always_apply=True),
    # A.GaussianBlur(),
    # A.ColorJitter(),
    # A.GaussNoise(),
])

<h2>Aux Functions</h2>

In [ ]:
def pad_frames(frames):
    target = np.zeros((FRAME_CAP, 3, IMAGE_WIDTH, IMAGE_HEIGHT))

    stopping_point = frames.shape[0]

    target[:stopping_point, :, :, :] = frames

    return target

In [ ]:
def truncate_frames(frames):
    return frames[:FRAME_CAP, :, :, :]

In [ ]:
def h_flip_img(img):
    global h_flip

    if h_flip >= 0.5:
        cv2.flip(img, 0, img)

In [ ]:
def v_flip_img(img):
    global v_flip

    if v_flip >= 0.5:
        cv2.flip(img, 1, img)

In [ ]:
def preprocess_image(image):
    global degree

    h_flip_img(image)
    v_flip_img(image)

    image = A.rotate(image, degree)
    
    image = transform(image=image)["image"]

    image = np.transpose(image, [2,0,1])

    image = np.expand_dims(image, 0)

    return image

In [ ]:
def preprocess_image_preview_version(image):
    global degree

    h_flip_img(image)
    v_flip_img(image)

    image = A.rotate(image, degree)
    
    image = transform_preview(image=image)["image"]

    image = np.expand_dims(image, 0)
    
    return image

In [ ]:
def save_as_video(frames):
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter('./tests/output.avi',fourcc, 10.0, (IMAGE_HEIGHT, IMAGE_WIDTH))
    
    for frame in frames:
        out.write(np.array(frame, dtype=np.uint8))
    out.release()

In [ ]:
def preview_frames(fps: int, src: str):
    global h_flip
    global v_flip
    global degree

    frames = torch.zeros((0,IMAGE_WIDTH,IMAGE_HEIGHT,3))
    count = 0

    vidcap = cv2.VideoCapture(src)

    success,image = vidcap.read()

    degree = random.randrange(-360,360)
    h_flip = random.random()
    v_flip = random.random()
    image = preprocess_image_preview_version(image)

    frames = np.vstack([frames, image])
    input_mask = np.zeros((FRAME_CAP), dtype=np.int8)

    makedirs("./tests", exist_ok=True)

    while success:
        vidcap.set(cv2.CAP_PROP_POS_MSEC,(count*fps))

        success,image = vidcap.read()

        if not success:
            break

        image = preprocess_image_preview_version(image)
        cv2.imwrite(f"./tests/{count}.jpg", image.squeeze())

        frames = np.vstack([frames, image])
        input_mask[count] = 1
        count += 1

    save_as_video(frames)

    print(frames)
    print(f"Frames Shape: {frames.shape}")
    print(input_mask)
    print(f"Input Masks Shape: {input_mask.shape}")

In [ ]:
def get_frames(fps: int, src: str, i: int):

    global h_flip
    global v_flip
    global degree

    frames = torch.zeros((0,3,IMAGE_WIDTH,IMAGE_HEIGHT))
    count = 0
    vidcap = cv2.VideoCapture(src)
    frame_count = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))

    if frame_count == 0:
        return None, None

    success,image = vidcap.read()

    degree = random.randrange(-360,360)
    h_flip = random.random()
    v_flip = random.random()
    image = preprocess_image(image)

    frames = np.vstack([frames, image])
    input_mask = np.zeros((FRAME_CAP), dtype=np.int8)

    while success:
        vidcap.set(cv2.CAP_PROP_POS_MSEC,(count*fps))
        success,image = vidcap.read()

        if not success:
            break

        image = preprocess_image(image)
        frames = np.vstack([frames, image])
        input_mask[count] = 1

        count += 1
        
    if frames.shape[0] > FRAME_CAP:
        raise Exception
    
    elif frames.shape[0] < FRAME_CAP:
        frames = pad_frames(frames)

    return frames, input_mask

In [ ]:
def create_record(files):
    files = [str(x) for x in files]
    return ",".join(files)

In [ ]:
def write_csv(records, save_dir):
    with open(save_dir, 'w', newline="\n") as myfile:
        wr = csv.writer(myfile, delimiter='\n', quotechar="", quoting=csv.QUOTE_NONE)
        wr.writerow(records)
        myfile.close()

In [ ]:
def preprocess_batch(start_file_idx: int, end_file_idx: int):

    train_records = []
    val_records = []

    train_records.append("file_name,og_file,pin")
    val_records.append("file_name,og_file,pin")

    file_names = [f for f in listdir(src_dir) if isfile(join(src_dir, f))]

    holdout_set = set()

    ran_once = False

    while True:
        
        for i, file_name in enumerate(file_names):
            
            pin = file_name.split("_")[0]

            file_name = join(src_dir, file_name)
            
            frames, input_mask = get_frames(FPS, f"{file_name}", i)

            if frames is None:
                continue

            if not ran_once:
                is_in_holdout = random.random()

                if is_in_holdout <= 0.2:
                    holdout_set.add(i)

            if i in holdout_set:
                dump_dir = dest_dir_val
                records = val_records
            else:
                dump_dir = dest_dir_train
                records = train_records

            record = create_record([f"{dump_dir}{start_file_idx}", file_name, pin])
            records.append(record)

            frames = frames.transpose([1,0,2,3])
            frames = np.float32(frames)

            data = {"image": frames, "mask": input_mask}
            hickle.dump(data, f"{dump_dir}{start_file_idx}", compression='gzip')

            print(f"File {start_file_idx} saved at: {dump_dir}{start_file_idx}")

            start_file_idx += 1

            if start_file_idx % 10 == 0:
                clear_output(True)
            
            if start_file_idx == end_file_idx:
                break
        
        if start_file_idx == end_file_idx:
            break

        ran_once = True

    write_csv(train_records, f"{dest_dir}train.csv")
    write_csv(val_records, f"{dest_dir}val.csv")

    print()

In [ ]:
def save_video_frames(dir, file_idx):
    frames, input_mask = get_frames(FPS, f"{dir}")

    frames = frames.transpose([1,0,2,3])
    frames = np.float32(frames)

    data = {"image": frames, "mask": input_mask}
    hickle.dump(data, f"{file_idx}", compression='gzip')

<h2>Runtime</h2>

In [ ]:
preview_frames(FPS, "/home/tyler/Downloads/NumaGuard-main/data/videos/4021_20240402_160547_748042.mp4")

In [ ]:
p1 = Process(target=preprocess_batch, args=(0, 5))
p1.start()

In [ ]:
p2 = Process(target=preprocess_batch, args=(5, 10))
p2.start()